In [ ]:
import warnings

# Filter out the specific warning
warnings.filterwarnings("ignore", message=".*`PyDataset` class.*")

# Loading data

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def get_generator(parent_folder, batch_size=32):
    train_data_dir = os.path.join(parent_folder, 'train')
    validation_data_dir = os.path.join(parent_folder, 'validation')
    test_data_dir = os.path.join(parent_folder, 'test')

    img_width, img_height = 224, 224

    # Define data augmentation parameters
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # For validation and test data, only rescale (no augmentation)
    val_test_datagen = ImageDataGenerator(rescale=1./255)

    # Generator for loading and augmenting training data
    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical'  # Assuming you're performing classification
    )

    # Generator for loading validation data
    validation_generator = val_test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical'
    )

    # Generator for loading test data
    test_generator = val_test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical'
    )

    return train_generator, validation_generator, test_generator

# Example usage:
parent_folder = 'F:/thesis/data'
train_generator, validation_generator, test_generator = get_generator(parent_folder)

# Building model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

def build_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape based on image dimensions
input_shape = (224, 224, 3)  # Assuming RGB images

# Define number of classes
num_classes = 38  # Assuming you have 38 classes

# Build the model
model = build_model(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

# Train

In [ ]:
# Define number of epochs
epochs = 100

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)